In [1]:
### Web Scraping Stack Overflow - Part II - Multiple pages

### Base code modified and expanded from
### https://medium.com/better-programming/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper-4e2c641cff8

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as bs
import requests

%matplotlib inline

In [3]:
#user_span_single = user_spans[0]

def user_find(user_span_single):
    
    trial = user_span_single.select('a')
    
    if len(trial) >= 1:
    
        intermediate = trial[0].text
        
        if '\r\n' in intermediate:
        
            return None
        
        return intermediate
    
    if 'anon' in user_span_single.text:
    
        return 'anonymous'
    
    else:
        return user_span_single.get_text(strip=True)

In [4]:
#time_spans = body.select("span.relativetime")
#time_stamp = [i["title"] for i in time_spans]

def time_find(users, time_stamp):
 
    polished_times = []
    times_iter = iter(time_stamp)

    for user_id in users:
        if user_id == 'community wiki' or user_id == 'anonymous':
            polished_times.append('None') 
        
        else:
            polished_times.append(next(times_iter))
            
    
    return polished_times

In [5]:
questions = []
summaries = []
tags = []
times = []
users = []
no_of_votes = []
no_of_answers = []

In [6]:
pages = np.arange(1, 20, 1)

In [7]:
for page in pages:
    
    #def get_top_questions(page):
    # WARNING: Only enter one of these 3 values [15, 30, 50].
    # Since, stackoverflow, doesn't display any other size questions list
    #url = url + "?sort=votes&pagesize={}".format(question_count)
    
    # Using requests module for downloading webpage content
    page = requests.get("https://stackoverflow.com/questions?tab=votes&page=" + str(page))

    # Parsing html data using BeautifulSoup
    soup = bs(page.content, 'html.parser')
    body = soup.find('body')

    # Extracting Top Questions
    question_links = body.select("h3 a.question-hyperlink")
    #error_checking(question_links, question_count)                     # Error Checking
    questions.extend([i.text for i in question_links])                      # questions list
    
    # Extracting Summary
    summary_divs = body.select("div.excerpt")
    #error_checking(summary_divs, question_count)                       # Error Checking
    summaries.extend([i.text.strip() for i in summary_divs])                 # summaries list
    
    # Extracting Tags
    tags_divs = body.select("div.summary > div:nth-of-type(2)")
    
    #error_checking(tags_divs, question_count)                          # Error Checking
    a_tags_list = [i.select('a') for i in tags_divs]                   # tag links
    
    tagged_tags = []

    for a_group in a_tags_list:
            tagged_tags.append([a.text for a in a_group])                         # tags list
    
    tags.extend(tagged_tags)
    
    # Extracting User Info
    user_spans = body.select("div.user-details")
    user_found = [user_find(user_span_single) for user_span_single in user_spans]
    users_list = [x for x in user_found if x]
    users.extend([x for x in user_found if x])
    print(len(users))
    
    # Extracting Question Time Stamps
    time_spans = body.select("span.relativetime")
    time_stamp = [i["title"] for i in time_spans]
    times.extend(time_find(users_list, time_stamp))
    
    #print(len(time_spans))                                             # time list
    
    # Extracting Number of votes
    vote_spans = body.select("span.vote-count-post strong")
    #error_checking(vote_spans, question_count)                         # Error Checking
    no_of_votes.extend([int(i.text) for i in vote_spans])                    # votes list
    
    # Extracting Number of answers
    answer_divs = body.select("div.status strong")
    #error_checking(answer_divs, question_count)                        # Error Checking
    no_of_answers.extend([int(i.text) for i in answer_divs])                 # answers list
    
# Putting all of them together
test_df = pd.DataFrame({'question': questions, 
                        'summary': summaries, 
                        'tags': tags,
                        'time_stamp': times,
                        'user_id': users,
                        'no_of_votes': no_of_votes,
                        'no_of_answers': no_of_answers})



50
100
150
200
250
300
350
400
450
499
549
599
649
699
749
799
849
899
949


ValueError: arrays must all be same length

In [8]:
len(times)

949

In [9]:
len(users)

949

In [10]:
len(no_of_answers)

950

In [ ]:
test_df.to_csv('test_scrape.csv')

In [ ]:
test_df.shape

In [ ]:
test_df.tail(56)